In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [3]:
left_nodes = clf.tree_.children_left[clf.tree_.children_left>0]
right_nodes = clf.tree_.children_right[clf.tree_.children_right>0]
node_indicator = clf.decision_path(X)
path_list = []
for i, j in enumerate(X):
    path_list.append(node_indicator.indices[node_indicator.indptr[i]:node_indicator.indptr[i+1]])

## Convert path to strings
path_column = np.array([])
for i, j in enumerate(X):
    path_as_string = []
    for node in path_list[i]:
        if node == 0:
            path_as_string.append('S')
        elif node in left_nodes:
            path_as_string.append('L')
        elif node in right_nodes:
            path_as_string.append('R')
            
    path_as_string.append('E')
    path_as_string = ' '.join(path_as_string)
    path_column = np.append(path_column, path_as_string)

In [4]:
chars = ['S', 'L', 'R', 'E']
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Xnew = np.hstack((X, path_column.reshape(-1,1)))
path_sequence = Xnew[:,4]
data = pd.DataFrame(Xnew)
data[5]=y
df = data.sample(frac=1).reset_index(drop=True)

# prepare dataset for training
get_path_lengths = lambda t: len(t.split())
paths_lengths = np.array([get_path_lengths(xi) for xi in path_sequence])

In [5]:
vocab_size = 4
label_size = 3
feature_size = 4
# cut the text in semi-redundant sequences of maxlen characters
maxlen = np.max(paths_lengths)
sentences = []
next_chars = []
features = []
labels = []

for i in range(0, len(df)):
    # get the feature
    curr_feat = np.array([df.iloc[i,0:4]])
    curr_path = df.iloc[i,4].split()
    curr_path_len = len(curr_path)
    curr_label = y[i]
    for j in range(1,curr_path_len):
        features.append(curr_feat)
        labels.append(curr_label)
        sentences.append(curr_path[0:j])
        next_chars.append(curr_path[j])
print('Vectorization...')

x_sent = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.bool)
x_feat = np.zeros((len(sentences), feature_size), dtype=np.float)
y_chars = np.zeros((len(sentences), vocab_size), dtype=np.bool)
y_feat = np.zeros((len(sentences), label_size), dtype=np.float)
#from keras.utils import to_categorical
#y_feat_tmp = to_categorical(df[5])

Vectorization...


In [6]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_sent[i, t, char_indices[char]] = 1
    y_chars[i, char_indices[next_chars[i]]] = 1
    x_feat[i,:] = features[i]
    y_feat[i,labels[i]]=1

In [7]:
index = 10
print(y_chars[index],y_feat[index],x_sent[index],x_feat[index])
print(y_chars.shape,y_feat.shape,x_sent.shape,x_feat.shape)

[False False False  True] [1. 0. 0.] [[ True False False False]
 [False False  True False]
 [False False  True False]
 [False  True False False]
 [False  True False False]
 [False False False False]
 [False False False False]] [6.  3.  4.8 1.8]
(560, 4) (560, 3) (560, 7, 4) (560, 4)


In [23]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten

h1_size = 5
latent_dim = 5

input_x_features = Input(shape=(feature_size,),name='ip_x')
hidden_state_x = Dense(h1_size, activation='relu',name='hidden_x')(input_x_features)
output_labels = Dense(3, activation='softmax',name='op_x')(hidden_state_x)

input_sent_features = Input(shape=(maxlen,vocab_size),name='ip_sent')
decoder = LSTM(latent_dim,return_state=False,return_sequences=False,name='lstm_sent')
decoder_outputs = decoder(input_sent_features)

merge_layer = concatenate([hidden_state_x,decoder_outputs],name='cat')
output_chars = Dense(vocab_size, activation='softmax',name='op_sent')(merge_layer)
model = Model([input_x_features,input_sent_features], [output_labels,output_chars])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ip_x (InputLayer)               (None, 4)            0                                            
__________________________________________________________________________________________________
ip_sent (InputLayer)            (None, 7, 4)         0                                            
__________________________________________________________________________________________________
hidden_x (Dense)                (None, 5)            25          ip_x[0][0]                       
__________________________________________________________________________________________________
lstm_sent (LSTM)                (None, 5)            200         ip_sent[0][0]                    
__________________________________________________________________________________________________
op_x (Dens

In [24]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten, GRU

h1_size = 5
latent_dim = 5

input_x_features = Input(shape=(feature_size,),name='ip_x')
hidden_state_x = Dense(h1_size, activation='relu',name='hidden_x')(input_x_features)
output_labels = Dense(3, activation='softmax',name='op_x')(hidden_state_x)

input_sent_features = Input(shape=(maxlen,vocab_size),name='ip_sent')
decoder = GRU(latent_dim,return_state=False,return_sequences=False,name='lstm_sent')
decoder_outputs = decoder(input_sent_features,initial_state=hidden_state_x)

#merge_layer = concatenate([hidden_state_x,decoder_outputs],name='cat')
output_chars = Dense(vocab_size, activation='softmax',name='op_sent')(decoder_outputs)
model = Model([input_x_features,input_sent_features], [output_labels,output_chars])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ip_x (InputLayer)               (None, 4)            0                                            
__________________________________________________________________________________________________
hidden_x (Dense)                (None, 5)            25          ip_x[0][0]                       
__________________________________________________________________________________________________
ip_sent (InputLayer)            (None, 7, 4)         0                                            
__________________________________________________________________________________________________
lstm_sent (GRU)                 (None, 5)            150         ip_sent[0][0]                    
                                                                 hidden_x[0][0]                   
__________

In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([x_feat,x_sent],[y_feat,y_chars],batch_size =20, epochs = 200,verbose=1)

Epoch 1/200
560/560 [==============================] - 1s 3ms/step - loss: 1.7422 - op_x_loss: 1.1171 - op_sent_loss: 0.6251 - op_x_acc: 0.3071 - op_sent_acc: 0.7411
Epoch 2/200
560/560 [==============================] - 0s 403us/step - loss: 1.7316 - op_x_loss: 1.1110 - op_sent_loss: 0.6207 - op_x_acc: 0.3161 - op_sent_acc: 0.7339
Epoch 3/200
560/560 [==============================] - 0s 425us/step - loss: 1.7298 - op_x_loss: 1.1095 - op_sent_loss: 0.6203 - op_x_acc: 0.3393 - op_sent_acc: 0.7446
Epoch 4/200
560/560 [==============================] - 0s 431us/step - loss: 1.7230 - op_x_loss: 1.1088 - op_sent_loss: 0.6142 - op_x_acc: 0.3036 - op_sent_acc: 0.7446
Epoch 5/200
560/560 [==============================] - 0s 424us/step - loss: 1.7175 - op_x_loss: 1.1070 - op_sent_loss: 0.6106 - op_x_acc: 0.3375 - op_sent_acc: 0.7518
Epoch 6/200
560/560 [==============================] - 0s 410us/step - loss: 1.7148 - op_x_loss: 1.1059 - op_sent_loss: 0.6089 - op_x_acc: 0.3339 - op_sent_acc: 0

560/560 [==============================] - 0s 674us/step - loss: 1.4951 - op_x_loss: 1.0882 - op_sent_loss: 0.4070 - op_x_acc: 0.3839 - op_sent_acc: 0.8357
Epoch 50/200
560/560 [==============================] - 0s 458us/step - loss: 1.4887 - op_x_loss: 1.0865 - op_sent_loss: 0.4022 - op_x_acc: 0.3964 - op_sent_acc: 0.8446
Epoch 51/200
560/560 [==============================] - 0s 583us/step - loss: 1.4815 - op_x_loss: 1.0875 - op_sent_loss: 0.3940 - op_x_acc: 0.3750 - op_sent_acc: 0.8464
Epoch 52/200
560/560 [==============================] - 0s 729us/step - loss: 1.4795 - op_x_loss: 1.0866 - op_sent_loss: 0.3929 - op_x_acc: 0.3946 - op_sent_acc: 0.8500
Epoch 53/200
560/560 [==============================] - 0s 821us/step - loss: 1.4739 - op_x_loss: 1.0880 - op_sent_loss: 0.3859 - op_x_acc: 0.3875 - op_sent_acc: 0.8554
Epoch 54/200
560/560 [==============================] - 1s 981us/step - loss: 1.4663 - op_x_loss: 1.0864 - op_sent_loss: 0.3800 - op_x_acc: 0.4143 - op_sent_acc: 0.8607

560/560 [==============================] - 0s 495us/step - loss: 1.2961 - op_x_loss: 1.0845 - op_sent_loss: 0.2116 - op_x_acc: 0.3839 - op_sent_acc: 0.9304
Epoch 97/200
560/560 [==============================] - 0s 462us/step - loss: 1.2935 - op_x_loss: 1.0835 - op_sent_loss: 0.2100 - op_x_acc: 0.4018 - op_sent_acc: 0.9321
Epoch 98/200
560/560 [==============================] - 0s 487us/step - loss: 1.2957 - op_x_loss: 1.0857 - op_sent_loss: 0.2100 - op_x_acc: 0.4000 - op_sent_acc: 0.9339
Epoch 99/200
560/560 [==============================] - 0s 649us/step - loss: 1.2829 - op_x_loss: 1.0831 - op_sent_loss: 0.1998 - op_x_acc: 0.4018 - op_sent_acc: 0.9286
Epoch 100/200
560/560 [==============================] - 0s 456us/step - loss: 1.2903 - op_x_loss: 1.0862 - op_sent_loss: 0.2040 - op_x_acc: 0.4036 - op_sent_acc: 0.9446
Epoch 101/200
560/560 [==============================] - 0s 590us/step - loss: 1.2863 - op_x_loss: 1.0840 - op_sent_loss: 0.2023 - op_x_acc: 0.4089 - op_sent_acc: 0.93

560/560 [==============================] - 0s 782us/step - loss: 1.2082 - op_x_loss: 1.0867 - op_sent_loss: 0.1215 - op_x_acc: 0.3875 - op_sent_acc: 0.9696
Epoch 145/200
560/560 [==============================] - 0s 644us/step - loss: 1.2006 - op_x_loss: 1.0805 - op_sent_loss: 0.1201 - op_x_acc: 0.4107 - op_sent_acc: 0.9643
Epoch 146/200
560/560 [==============================] - 0s 659us/step - loss: 1.2014 - op_x_loss: 1.0837 - op_sent_loss: 0.1177 - op_x_acc: 0.4125 - op_sent_acc: 0.9679
Epoch 147/200
560/560 [==============================] - 0s 551us/step - loss: 1.1917 - op_x_loss: 1.0811 - op_sent_loss: 0.1106 - op_x_acc: 0.4125 - op_sent_acc: 0.9714
Epoch 148/200
560/560 [==============================] - 0s 812us/step - loss: 1.2003 - op_x_loss: 1.0818 - op_sent_loss: 0.1185 - op_x_acc: 0.4125 - op_sent_acc: 0.9661
Epoch 149/200
560/560 [==============================] - 0s 468us/step - loss: 1.1931 - op_x_loss: 1.0809 - op_sent_loss: 0.1122 - op_x_acc: 0.4161 - op_sent_acc: 0

560/560 [==============================] - 0s 689us/step - loss: 1.1651 - op_x_loss: 1.0803 - op_sent_loss: 0.0848 - op_x_acc: 0.4071 - op_sent_acc: 0.9732
Epoch 193/200
560/560 [==============================] - 0s 641us/step - loss: 1.1669 - op_x_loss: 1.0805 - op_sent_loss: 0.0864 - op_x_acc: 0.3982 - op_sent_acc: 0.9786
Epoch 194/200
560/560 [==============================] - 0s 882us/step - loss: 1.1651 - op_x_loss: 1.0791 - op_sent_loss: 0.0860 - op_x_acc: 0.4089 - op_sent_acc: 0.9679
Epoch 195/200
560/560 [==============================] - 0s 675us/step - loss: 1.1549 - op_x_loss: 1.0781 - op_sent_loss: 0.0768 - op_x_acc: 0.4036 - op_sent_acc: 0.9768
Epoch 196/200
560/560 [==============================] - 0s 746us/step - loss: 1.1588 - op_x_loss: 1.0778 - op_sent_loss: 0.0809 - op_x_acc: 0.4000 - op_sent_acc: 0.9821
Epoch 197/200
560/560 [==============================] - 0s 512us/step - loss: 1.1627 - op_x_loss: 1.0781 - op_sent_loss: 0.0846 - op_x_acc: 0.4018 - op_sent_acc: 0

In [10]:
def sample(x):
    n = x.shape[0]
    x_f = x.reshape(1,feature_size)
    token = 'S'
    cont = True
    text = [token]
    x_sent = np.zeros((1,maxlen,vocab_size),dtype=np.bool)
    x_sent[0,0,char_indices[token]] = 1
    label = []
    index = 1
    while cont & (index <maxlen):
        pred = model.predict([x_f.reshape(1,feature_size),x_sent])
        char_index = np.argmax(pred[1])
        label.append(np.argmax(pred[0])) 
        x_sent[0,index,char_index] = 1
        next_char = indices_char[char_index]
        text.append(next_char)
        index += 1    
        if next_char == 'E':
            cont = False
    return [text,label]

In [31]:
count = []
for i in range(10):
    curr_feat = np.array([df.iloc[i,0:4]])
    path,label= sample(curr_feat)
    print('actual vs predicted: ', df.iloc[i,4] ,' vs ', ' '.join(path), 'labels: ', df.iloc[i,5],label[0])
    count.append(df.iloc[i,5]==label[0])
np.mean(count)

actual vs predicted:  S R R R E  vs  S R R R E labels:  2 0
actual vs predicted:  S L E  vs  S L E labels:  0 1
actual vs predicted:  S R R L L E  vs  S R R L L E labels:  2 0
actual vs predicted:  S R L L R E  vs  S R L L L E labels:  2 0
actual vs predicted:  S R R L R E  vs  S R R L L E labels:  1 0
actual vs predicted:  S R L L L E  vs  S R L L L E labels:  1 1
actual vs predicted:  S L E  vs  S L E labels:  0 1
actual vs predicted:  S R R R E  vs  S R R R E labels:  2 0
actual vs predicted:  S L E  vs  S L E labels:  0 1
actual vs predicted:  S R R R E  vs  S R R R E labels:  2 0


0.1